# Data management & Univariate analysis

In this assignment I will start data analysis for the research project described here:

[Research proposal](https://github.com/CourseraParticipant/Data-Management-and-Visualization/blob/master/Research%20proposal-checkpoint.ipynb)

More precisely, I shall conduct data management & univariate analysis. The goals will be done in few steps, and the code in Python will be thus splitted into blocks. Let me start by loading the data set NESARC.

In [1]:
# Step 1: Loading data which is saved in the working directory as csv file under name "nesarc.csv"
%matplotlib inline  
# specific for jupyter notebook: for graphs to be displayed inline
# Importing libraries
import pandas
import numpy
# load data set
data = pandas.read_csv("nesarc.csv",low_memory = False)
# Checking how many observations the dataset contains
print("Total number of rows in the data source")
print(len(data))
#Checking how many variables/columns are traked with tha data
print("Total number of variables present in the original data")
print(len(data.columns))

Total number of rows in the data source
43093
Total number of variables present in the original data
3008


The next step is extract a subset from  the original data, which includes only observations relevant for my research question. Namely, according to my research focus, I am interesting only into the following 4 variables: S1Q1F, S1Q8A, S1Q9B and S13Q1. Thus, I will subset the original data containing only those 4 variables / columns. Each of these 3 variables will be transformed in numeric and missing data will be set to NaN.

In [2]:
# Step 2: Data management: Extracting only needed data & Conversion to numeric & Setting missing data to NaN
# Firstly, since Python is case-sensitive I shall change the names of all columns to uppercase
data.columns = map(str.upper,data.columns)
# extracting only 4 necessary variables:
sub0=data[['S1Q1F','S1Q8A', 'S1Q9B', 'S13Q1']]
sub1 =sub0.copy()
#set missing data to NaN
sub1['S1Q9B']=sub1['S1Q9B'].replace(' ', numpy.nan)
sub1['S1Q1F']=sub1['S1Q1F'].replace(9, numpy.nan)
sub1['S13Q1']=sub1['S13Q1'].replace(99, numpy.nan)

# conversion to numeric
sub1['S1Q1F']=pandas.to_numeric(sub1['S1Q1F'])
sub1['S1Q8A']=pandas.to_numeric(sub1['S1Q8A'])
sub1['S1Q9B']=pandas.to_numeric(sub1['S1Q9B'])
sub1['S13Q1']=pandas.to_numeric(sub1['S13Q1'])


According to my research focus, I am interesting only into subjects born in the U.S.A. As the codebook states, the variable "S1Q1F" keeps track on the question whether a subject was /was not burn in the United States.
Before extracting the subset of data, let us now look at the distribution of this variable.

In [3]:
# 1st univariate analysis: How many subjects in data are born in the U.S.?
print("Counts of U.S. born / non-born data subjects: 1 - born in the U.S., 2 - not born in the U.S., NaN - Unknown")
c1 = sub1['S1Q1F'].value_counts(sort = False,dropna = False)
print(c1)
# Now the same counts but in terms of frequencies/percentages
print("Frequency of U.S. born / non-born data subject: 1 - born in the U.S.,2 - not born in the U.S., 9 - Unknown")
p1 = sub1['S1Q1F'].value_counts(sort = False,dropna = False, normalize = True)
print(p1)

Counts of U.S. born / non-born data subjects: 1 - born in the U.S., 2 - not born in the U.S., NaN - Unknown
NaN       151
 1.0    35622
 2.0     7320
Name: S1Q1F, dtype: int64
Frequency of U.S. born / non-born data subject: 1 - born in the U.S.,2 - not born in the U.S., 9 - Unknown
NaN     0.003504
 1.0    0.826631
 2.0    0.169865
Name: S1Q1F, dtype: float64


 **Summary for the first frequency check -  U.S. born subjects** : Here we see that 35622 or ca. 82,7% of the study subjects were born in the U.S., 7320 subjects were not born in the U.S. whereas for 151 individuals the data does not contain information about birth of place.
 
 Since I aim to investigate only  U.S. born subjects, I may conclude that my subset will be still sufficiently large (as it contains 35622 or 82.7 % of the provided data).
 Let me know create my data set "myData" by selecting  only subjects born in the U.S.

In [4]:
# Making my own data containing only subjects who were born in the U.S.
sub2 =sub1[sub1['S1Q1F']==1]
myData = sub2.copy()
# Double-check : Hereby I want to check whether myData really (quantitatively) selected target group. I will check the 
# number of observations in myData and compare it with the above obtained counts. According to the counts,there should be
# 35622 rows in the subset
print ('Double-check for the subset of data: Total number of rows in my selected data')
print (len(myData))

Double-check for the subset of data: Total number of rows in my selected data
35622


As stated in my research proposal, I want to define a new variable named EMPL_TYPE which is a combination of the variables describing employments status and last job's type (S1Q8A and  S1Q9B, respectively). Indeed, EMPL_TYPE takes value 0 (Unemployed) if and only if S1Q8A = 2. Otherwise, EMPL_TYPE  takes identical value as the variable S1Q9B. Let me define this variable in the following block of the code.
    

In [5]:
# defining new variable EMPL_TYPE. It will be defined in two steps. 
# 1st step of the definition: Define ne EMPL_TYPE as a copy of the variable S1Q9B
myData['EMPL_TYPE']=myData['S1Q9B']
#2nd step of definition: At the moment, EMPL_TYPE is just a copy of the S1Q9B. This is not right only in the cases when
# S1Q8A = 2 (that is, the subject is unemployed). In this case, I want that my EMPL_TYPE takes value 0. Let me change 
# its value for this particulr case
myData.loc[myData['S1Q8A']==2,'EMPL_TYPE']=0

In the following code block I perform the last data management step. It is about the variable which counts number of times stayed in hospital overnight in last 12 months (variable S13Q1). Currently, it may take values between 0 and 98.Actually, I could define one new variable HOSP_MONTHLY which estimates how many times per month (on average) the person has visited hospital in last 12 months. Thus, its value is simply set to S13Q1/12. Thus, the new variable HOSP_MONTHLY may take values between 0 and 8

In [6]:
# define new variable HOSP_MONTHLY as number of average monthly hospital stays in past 12 months
myData['HOSP_MONTHLY']= round(myData['S13Q1'] / 12)

In [7]:
# Step 3 : Univariate analysis on the subset
# The 1st variable - Employment status in the last 12 months, represented by 'S1Q8A' with codes: 1 - Yes, 2 - No
# Counts of employed/unemployed subjects (in last 12 moths)
print('Counts for employed/unemployed subjects who were born in the U.S.: 1 - employed, 2 - unemployed')
c2=myData['S1Q8A'].value_counts(sort = False, dropna = False)
print(c2)
print('Associated percentages for employed/unemployed subjects who were born in the U.S.: 1- employed, 2-unemployed')
p2=myData['S1Q8A'].value_counts(sort = False, dropna = False, normalize = True)
print(p2)

Counts for employed/unemployed subjects who were born in the U.S.: 1 - employed, 2 - unemployed
1    25301
2    10321
Name: S1Q8A, dtype: int64
Associated percentages for employed/unemployed subjects who were born in the U.S.: 1- employed, 2-unemployed
1    0.710263
2    0.289737
Name: S1Q8A, dtype: float64


 **Summary for the second frequency check - Employment status in the last 12 months of  U.S. born subjects ** : Here we see that 25301 out of 35622, or approximately 71% of the U.S. born subjects, have been employed in the last 12 months, whereas 10321 U.S. born subjects or ca. 29 % were unemployed in the same time period. 
 
 Let me now look at the second variable of the interest: the occupation type, which is given by the variable 'EMPL_TYPE'. 

In [8]:
# The second variable - Employment type represented by 'EMPL_TYPE' 
# Counts of occupation types
print('Counts for employment types of  subjects who were born in the U.S.:')
c3=myData['EMPL_TYPE'].value_counts(sort = True, dropna = False)
print(c3)
# I set sort = TRUE as codes themselves have no meaning as numbers. And by sort = true, I can see which
# occupation type occurs the most frequent, which the least  etc.
print('Associated percentages for employment categories of subjects who were born in the U.S.: ')
#frequencies of the occupations
p3=myData['EMPL_TYPE'].value_counts(sort = True, dropna = False, normalize = True)
print(p3)
print ('Codes for the occupation type:')
print ('0 - Unemployed in the last 12 months')
print ('1 - Executive, Administrative, and Managerial')
print('2 - Professional Speciality')
print('3 - Technical and Related Support')
print('4 - Sales')
print('5 - Administrative Support, including Clerical')
print(' 6 - Private Household')
print ('7 - Protective Services')
print ('8 - Other Services')
print(' 9 - Farming, Forestry and Fishing')
print('10 - Precision Production, Craft and Repair')
print('11 - Operators, Fabricators and Laborers')
print('12 - Transportation and Material Moving')
print('13 - Handlers, Equipment Cleaners and Laborers')
print('14 - Military')
print ('BL - NA, never worked for pay or in family business or farm')

Counts for employment types of  subjects who were born in the U.S.:
0.0     10321
2.0      4836
1.0      3866
8.0      3587
4.0      2605
5.0      2504
3.0      2303
11.0     1808
10.0      924
12.0      764
13.0      753
7.0       461
9.0       368
14.0      278
6.0       244
Name: EMPL_TYPE, dtype: int64
Associated percentages for employment categories of subjects who were born in the U.S.: 
0.0     0.289737
2.0     0.135759
1.0     0.108528
8.0     0.100696
4.0     0.073129
5.0     0.070294
3.0     0.064651
11.0    0.050755
10.0    0.025939
12.0    0.021447
13.0    0.021139
7.0     0.012941
9.0     0.010331
14.0    0.007804
6.0     0.006850
Name: EMPL_TYPE, dtype: float64
Codes for the occupation type:
0 - Unemployed in the last 12 months
1 - Executive, Administrative, and Managerial
2 - Professional Speciality
3 - Technical and Related Support
4 - Sales
5 - Administrative Support, including Clerical
 6 - Private Household
7 - Protective Services
8 - Other Services
 9 - Farming, For

 **Summary for the third frequency check - Employment type of  U.S. born subjects ** : Firstly, note that the previous frequency check and the current one agree on the number of uneployed which is 10321 subjects, and it is the biggest category of the EMPl_TYPE variable. Here we see the most of employed subjects belong to category "professional speciality" (4836 persons or 13,6% of U.S. born subjects), followed by " Executive, Administrative, and Managerial" (10,9% or 3866 subjects) and then "Other services" (10.1% or 3587 people). The least popular occupation type among subjects, according to the data, is "Privite haousehold" with only 244 subjects. The second smallest category is "Military" group with 278 persons.
 
 Let me now look at the last variable of the interest: the average number of monthly hospital stays (in past 12 months), which is given by the variable 'HOSP_MONTHLY'. 

In [9]:
# The third variable -  Number of average monthly times stayed in hospital in last 12 months 
# (excluding delivery of healty liveborn infant)
# Variable name: 'HOSP_MONTHLY'
myData['HOSP_MONTHLY']=pandas.to_numeric(myData['HOSP_MONTHLY'])
# Counts of overnight avg. monthly hospital stays
print('Counts of avg. monthly hospital stays overnight (last 12 months)')
c4=myData['HOSP_MONTHLY'].value_counts(sort = True, dropna = False)
print(c4)
# associated frequencies
print('Associated percentages for avf. monthly hospital stays overnight (last 12 months)')
p4=myData['HOSP_MONTHLY'].value_counts(sort = True, dropna = False, normalize = True)
print(p4)

Counts of avg. monthly hospital stays overnight (last 12 months)
 0.0    34695
NaN       873
 1.0       44
 2.0        7
 8.0        1
 5.0        1
 3.0        1
Name: HOSP_MONTHLY, dtype: int64
Associated percentages for avf. monthly hospital stays overnight (last 12 months)
 0.0    0.973977
NaN     0.024507
 1.0    0.001235
 2.0    0.000197
 8.0    0.000028
 5.0    0.000028
 3.0    0.000028
Name: HOSP_MONTHLY, dtype: float64


** Summary for the fourth frequency check - Overnight hospital stays (in last 12 months) **:  The last univariate analysis says that the most of subjects has average number of mothly hospital stay equal to zero within last 12 moths . More precisely,  34695 or almost 97.4% of all U.S. born citizens in the data set did not stay in the hospital. Less then 0,1% of subjects has stayed on avereage  once per month in the hospital.
Missing data: There were 873 subjects or almost 2.5% of all U.S. born subjects on which data is missing for this specific question.

# Summary

By investigating only U.S. born subjects, I approximately exclude 17.3% of all data provided in NESARC and analyze data on 35622 subjects.
71.9% of those 35622 subjects,  are /were employed within last 12 months. The most popular job among employed subjests,  seems to be categorized as "professional speciality" and the least popular category is "private household". Regarding the overnight stay in hospital,the big majority of subjects (34695 or 97.4%) did not stay in hospital (or stayed maximally 5 timies in total) in the last 12 months. 